In [27]:
import re

In [31]:
texto_tmp = "temp.txt"
texto = 'solo_espanol.txt'

tweets_limpio = "new_tweets.txt"

new_tweets = open(tweets_limpio,'w+',encoding='utf-8')

twitter_username_re = re.compile(r'@([A-Za-z0-9_]+)')

with open(texto_tmp,'r', encoding='utf-8') as file:
    for line in file:
        #print(line)
        sentence =line.split(' ')
        for token in range(len(sentence)):
            if sentence[token].find('@') != -1:
                if re.match(twitter_username_re, sentence[token]):
                    sentence[token] = 'USUARIO'
            elif sentence[token].find('http') != -1:
                sentence[token] = 'LIGA'
                
        print(' '.join(sentence))
        new_tweets.write(' '.join(sentence))
        #tweets.append(list(enumerate(sentence)))
        print(list(enumerate(sentence)))        

new_tweets.close()
    

USUARIO cuando van a regresar el paquete perdido de nuestros amigos USUARIO USUARIO es

[(0, 'USUARIO'), (1, 'cuando'), (2, 'van'), (3, 'a'), (4, 'regresar'), (5, 'el'), (6, 'paquete'), (7, 'perdido'), (8, 'de'), (9, 'nuestros'), (10, 'amigos'), (11, 'USUARIO'), (12, 'USUARIO'), (13, 'es\n')]
¿Por qué no aprietan la pasta dental desde abajo? 😠😡 es

[(0, '¿Por'), (1, 'qué'), (2, 'no'), (3, 'aprietan'), (4, 'la'), (5, 'pasta'), (6, 'dental'), (7, 'desde'), (8, 'abajo?'), (9, '😠😡'), (10, 'es\n')]
"No es tu trabajo mejorar un mundo inherentemente caótico". es

[(0, '"No'), (1, 'es'), (2, 'tu'), (3, 'trabajo'), (4, 'mejorar'), (5, 'un'), (6, 'mundo'), (7, 'inherentemente'), (8, 'caótico".'), (9, 'es\n')]
#HarryStylesConExa Den todo de ustedes, yo voy a dar todo y más. ❤ caracterDeSaltoDeLínea #HarryStylesMexico USUARIO  caracterDeSaltoDeLínea (52)5543207368 es

[(0, '#HarryStylesConExa'), (1, 'Den'), (2, 'todo'), (3, 'de'), (4, 'ustedes,'), (5, 'yo'), (6, 'voy'), (7, 'a'), (8, 'dar'), (9, '